In [1]:
import IPython
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sympy
import datetime

# для расчетов над tdb
from pycalphad import Database, binplot

# для MCMC расчетов
import pymc as pm  # пакет для MCMC расчетов 
import arviz as az # пакет для работы с типом данных arviz
# import pytensor
import pytensor.tensor as pt
# import theano
# theano.config.exception_verbosity = 'high' # должно выдавать подробное описание ошибки, но не помогает

import aesara
import scipy

import seaborn as sns

# пути к tdb
db18 = Database('tdbs/CoCr-18Cac_with_new_functions.tdb')
db10 = Database('tdbs/CoCr-01Oik_with_new_functions.tdb')

# опытные данные
df_sigma_fcc = pd.read_excel('emp_data/sigma_fcc_allibert.xls')
df_sigma_hcp = pd.read_excel('emp_data/sigma_hcp_allibert.xls')
df_bcc = pd.read_excel('emp_data/sigma_bcc_allibert.xls')
# для 0.5
df_hcp_fcc = pd.concat([df_sigma_fcc, df_sigma_hcp])
df_hcp_fcc['T'] = df_hcp_fcc['T'].round(2)
df_hcp_fcc['cr_conc'] = df_hcp_fcc['cr_conc'].round(6)
df_hcp_fcc = df_hcp_fcc[(df_hcp_fcc['phase'] == 'sigma_old')].reset_index()
df_hcp_fcc.sort_values('T', inplace=True)
# для 0.75
df_bcc['T'] = df_bcc['T'].round(2)
df_bcc['cr_conc'] = df_bcc['cr_conc'].round(6)
df_bcc = df_bcc[(df_bcc['phase'] == 'sigma_old')].reset_index()
df_bcc.sort_values('T', inplace=True)

# трейсы
trace18 = az.from_json('calc_res/trace_cocr18_2Sx700x1000x2_20230509.json')
ppc18 = az.from_json('calc_res/ppc_cocr18_2Sx700x1000x2_20230509.json')
pp18 = az.from_json('calc_res/pp_cocr18_2Sx2000_20230509.json')
main18 = trace18.extend(ppc18)
par_list18 = ['GSCRCO1', 'GSCOCRCO1', 'GSCOCRCO2', 'GSCRCO2', 'GSCOCR1',  'GSCOCR2', 'GSCOCR3']

trace10 = az.from_json('calc_res/trace_cocr10_2Sx700x1000x4.json')
ppc10 = az.from_json('calc_res/ppc_cocr10_2Sx700x1000x4.json')
pp10 = az.from_json('calc_res/pp_cocr10_2Sx4000.json')
main10 = trace10.extend(ppc10)
par_list10 = ['SIGMA_OLD_COCRCO_0', 'SIGMA_OLD_COCRCO_1', 'SIGMA_OLD_COCRCR_0', 'SIGMA_OLD_COCRCR_1']


print(f"Running on PyMC v{pm.__version__}") # 5.3.1 or 5.1.2
print(f"Running on NumPy v{np.__version__}") # 1.22.1
print(f"Running on ArviZ v{az.__version__}") # 0.12.1

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Running on PyMC v5.3.1
Running on NumPy v1.22.1
Running on ArviZ v0.15.1


Функции

In [9]:
# запись словаря со значениями параметров по tdb
def from_piecewise_to_dict(db, parameters_list):
    import sympy
    full_par_dict = db.symbols
    tdb_par_dict = dict()

    for par in parameters_list:
        tdb_par_dict[par] = np.float32(full_par_dict[par].args_as_sympy()[0])
    return tdb_par_dict

# собираем цепи в одну для каждого параметра, словарь
def chains_concatenate(traces, parameters_list):
    # создаем словарь с распределениями параметров
    par_chains_dict = dict()
    for par in parameters_list:
        par_chains_dict[par] = np.float32(np.concatenate(traces.posterior.variables[par]))
    
    # вычисляем объем каждого сэмпла
    chains_num = traces.sample_stats.chain.data.shape
    draws_num = traces.sample_stats.draw.data.shape
    num_points = chains_num[0]*draws_num[0]
    
    return par_chains_dict

def chain_size(traces):
    # создаем словарь с распределениями параметров
    chains_num = traces.sample_stats.chain.data.shape
    draws_num = traces.sample_stats.draw.data.shape
    num_points = chains_num[0]*draws_num[0]
    
    return num_points

# для каждого распределения параметра считаем распределение квадратов отклонений от значения в tdb
def mse_dictribution_dict(par_chains_dict, tdb_par_dict, par_list):
    mse_par_dict = dict()

    for par in par_list:
        mse_par_dict[par] = np.float_power(np.subtract(par_chains_dict[par], tdb_par_dict[par]), 2)

    return mse_par_dict

# выдает словарь где для каждой температуры есть распределение предсказанных для нее значений
def from_ppc_to_temp_dict(ppc, var_name_str):
    yt_dict = dict()
    ppc_conc = np.concatenate(ppc.posterior_predictive.data_vars[var_name_str].data)
    t_num = ppc_conc.shape[1]
    p_num = ppc_conc.shape[0]
    
    for i in range(t_num):
        i_list = []
        
        for j in range(p_num):
            i_list.append(ppc_conc[j][i])

        yt_dict[i] = i_list # key = индекс температуры в T
    return yt_dict

# выдает словари с аопстериорной вероятностью каждого параметра и с логарифмом вероятности
def ppp_and_lnppp_dict_calc(di, par_list, mse_par_dict, num):
    import scipy

    ppp_dict = dict()
    ln_ppp_dict = dict()
    di = 0.000001
    # num = len(mse_par_dict[0])

    for par in par_list:
        list_cnt = [x for x in mse_par_dict[par] if x <= di]

        if len(list_cnt) == 0:
            print(par)
            mean = np.mean(mse_par_dict[par])
            std = np.std(mse_par_dict[par])
            # не подходит для любого параметра, только для норм распределнного
            ppp_dict[par] = scipy.stats.norm(mean, std).pdf(di)
        else:
            # print(par)
            # print(len(list_cnt))
            # print(num)
            ppp_dict[par] = len(list_cnt)/num

        ln_ppp_dict[par] = np.log(ppp_dict[par])

    return ln_ppp_dict, ppp_dict

# вычисление логарифма апостериорной вероятности модели на основе словаря
def model_pp_calc(par_list, ln_ppp_dict):
    ppp = 0

    for par in par_list:
        ppp += ln_ppp_dict[par]
    return ppp

Подготовительные вычисления

In [10]:
tdb_par_dict10 = from_piecewise_to_dict(db10, par_list10)
tdb_par_dict18 = from_piecewise_to_dict(db18, par_list18)

par_dict10 = chains_concatenate(trace10, par_list10)
par_dict18 = chains_concatenate(trace18, par_list18)

num10 = chain_size(trace10)
num18 = chain_size(trace18)

mse_par_dict10 = mse_dictribution_dict(par_dict10, tdb_par_dict10, par_list10)
mse_par_dict18 = mse_dictribution_dict(par_dict18, tdb_par_dict18, par_list18)

ln_ppp_dict10, ppp_dict10 = ppp_and_lnppp_dict_calc(0.000001,par_list10, mse_par_dict10, num10)
ln_ppp_dict18, ppp_dict18 = ppp_and_lnppp_dict_calc(0.000001,par_list18, mse_par_dict18, num18)

SIGMA_OLD_COCRCO_1
SIGMA_OLD_COCRCR_1


Сравнение по апостериорной вероятности

Вывод: 18 модель точнее

In [11]:
pp10 = model_pp_calc(par_list10, ln_ppp_dict10)
pp18 = model_pp_calc(par_list18, ln_ppp_dict18)
print(f'posterior probability of 10 model:{pp10}')
print(f'posterior probability of 18 model:{pp18}')

posterior probability of 10 model:-71.16073516665647
posterior probability of 18 model:-1.005843517326468


Сравнение по BF

Вывод: разницы между моделя почти нет..............

In [32]:
b = np.exp((trace10.log_likelihood.variables['y_norm_05'].max()
      +trace10.log_likelihood.variables['y_norm_75'].max())
-(trace18.log_likelihood.variables['y_norm_05'].max()
  +trace18.log_likelihood.variables['y_norm_75'].max()))
# bf = np.exp(b)
bf

<xarray.Variable ()>
array(0.00080603)

Сравнение по WAIC

10 модель лучше

In [47]:
t1075 = az.waic(trace10, var_name='y_norm_75', pointwise=True)
waic1075 = -2*t1075.elpd_waic
t1875 = az.waic(trace18, var_name='y_norm_75', pointwise=True)
waic1875 = -2*t1875.elpd_waic
print(f'waic for y_norm_75 model 10: {waic1075}')
print(f'waic for y_norm_75 model 18: {waic1875}')

waic for y_norm_75 model 10: 7047.67048888046
waic for y_norm_75 model 18: 261193005.57875085


c:\Users\emgebrishvili\AppData\Local\Programs\Python\Python310\lib\site-packages\arviz\stats\stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


10 модель снова лучше

In [49]:
t1005 = az.waic(trace10, var_name='y_norm_05', pointwise=True)
waic1005 = -2*t1005.elpd_waic
t1805 = az.waic(trace18, var_name='y_norm_05', pointwise=True)
waic1805 = -2*t1805.elpd_waic
print(f'waic for y_norm_75 model 10: {waic1005}')
print(f'waic for y_norm_75 model 18: {waic1805}')

waic for y_norm_75 model 10: 2405.2353845254406
waic for y_norm_75 model 18: 10629.097527046823


c:\Users\emgebrishvili\AppData\Local\Programs\Python\Python310\lib\site-packages\arviz\stats\stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(
